Datasæt: https://www.opendata.dk/city-of-copenhagen/monumenter

CSV: https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:monumenter&outputFormat=csv&SRSNAME=EPSG:4326

1) Hvor mange monumenter er der i København?

2) Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja

3) Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn?
	F.eks:
		def monumentById(monumentId):
			return coordinates
	
3.a) Vis monumentet som bliver returneret i metoden på et kort over københavn ved brug af plotting. (Se afsnittet om 'Folium and Bokeh' under notebooks/03-3 Plotting)

4) Find navnet på monumentet med x og y koordinaterne eller længde- og breddegraderne?
	
	x = 724407.424966
	y = 6175719.798486
	
	MULTIPOINT ((12.555485308174104 55.69383926601615))

5) Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.

6) Gør kortet interaktiv så navnet på monumenterne vises når man trykker på et plot. (Se afsnittet om 'Interactive plots with bokeh' under notebooks/03-3 Plotting)

#### MINIMUM BESVARELSE: 3 + 3.a + 5

In [ ]:
import pandas as pd
df = pd.read_csv('monumenter.csv')

In [ ]:
# --------------------------------------------
# 1) Hvor mange monumenter er der i København?
# --------------------------------------------

df_entries = len(df)
print('Antal monumenter (rækker):', df_entries, '\n')

# Frasortér alle monumenter uden navn
df_sorted = len(df[df['navn'].notnull()])
print('Antal monumenter u. navne:', df_sorted, '\n')

# (dotI)info angiver bl.a. også antal -> RangeIndex
#df.info()

In [ ]:
# -----------------------------------------------------------------------
# 2) Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja
# -----------------------------------------------------------------------

df_graffiti = len(df[df['graffitirenhold'] == 'Ja'])

print('Antal monumenter med graffitirenhold:', df_graffiti, '\n')

In [ ]:
# ---------------------------------------------------------------------------------------------------
# 3) Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn?
# ---------------------------------------------------------------------------------------------------

import re

# Vis alle data frame kolonner
#print(df.columns,'\n')

# Note-to-self: Oprindelig funktion
def monumentById(id):
    
    # Data frame indeholdende id, samt koordinater
    df_container = df[['web_id', 'wkb_geometry']]

    # Sæt index til at bruge monumentnavn
    df_index_by_id = df_container.set_index('web_id')

    # Vis koordinater ud fra id
    try:
        return df_index_by_id.loc[id]
    except:
        return print('Det valgte ID blev ikke fundet!')

# Note-to-self: Denne funktion er tilpasset til brug for kort marker
def monumentByName(name):

    # Data frame indeholdende navne, samt koordinater
    df_container = df[['navn', 'wkb_geometry']]
    
    # Sæt index til at bruge monumentnavn
    df_index_by_name = df_container.set_index('navn')
    
    # Vis koordinater ud fra navn
    try:
        # Gem multipoint som string
        multipoint = df_index_by_name.loc[name][0]
        #print('1', multipoint)
        
        # Vha. regex, find og list alle floats
        multipoint = re.findall('\d+\.\d+', multipoint)
        #print('2',multipoint)
        
        # Tilføj navn til liste
        multipoint.append(name)
        
        return multipoint
    except:
        return print('Det valgte navn blev ikke fundet!')
    
#Note-to-self: Denne funktion er lavet efter jeg så longitude og latitude som kolonner
def monumentByName2(name):

    # Data frame indeholdende navne, samt koordinater
    df_container = df[['navn', 'longitude', 'latitude']]
    
    # Sæt index til at bruge monumentnavn
    df_index_by_name = df_container.set_index('navn')

    # Vis koordinater ud fra navn
    try:
        # Gem multipoint som string
        multipoint = df_index_by_name.loc[name]
        
        return multipoint
    except:
        return print('Det valgte navn blev ikke fundet!')

#print(monumentById(343),"\n")
print(monumentByName('Spejldam'), '\n')
print(monumentByName2('Spejldam'))

In [ ]:
# ---------------------------------------------------------------------------------------------------
# 3.a) Vis monumentet som bliver returneret i metoden på et kort over københavn ved brug af plotting.
# ---------------------------------------------------------------------------------------------------

import json
import requests
import folium
import urllib

# Returnér længde- og breddegrad for København
def get_city_location(city='Copenhagen'):
    """Get the location coordinates from OpenStreetMaps"""
    url_nomatim_api = 'https://nominatim.openstreetmap.org/search'
    r = requests.get(url_nomatim_api, params={'format': 'json', 'city': city})
    results = r.json()
    #print('results: ',results)
    location = results[0]
    lat, lon = float(location['lat']), float(location['lon'])
    return lat, lon

# Opret kort
map_osm = folium.Map(location=(get_city_location()), zoom_start=10)

# Returnér længde- og breddegrad for Spejldams monumentet
#monument = monumentByName('Spejldam')
monument = monumentByName2('Spejldam')

# Tegn markør, inkl. interaktion for visning af monumentets navn
#folium.Marker(location=[monument[1], monument[0]], popup=monument[2]).add_to(map_osm)
folium.Marker(location=[monument[1], monument[0]], popup=monument.name).add_to(map_osm)

# Gem kortet til senere...
map_osm.save('./osm.html')

# Vis kort
map_osm

In [ ]:
# -----------------------------------------------------------------------------------
# 4) Find navnet på monumentet med x og y koordinaterne eller længde- og breddegraderne?
# x = 724407.424966
# y = 6175719.798486
# MULTIPOINT ((12.555485308174104 55.69383926601615))
# -----------------------------------------------------------------------------------

# x,y
df_xcoord = df[df['x'].isin(['724407.424966'])].navn
df_ycoord = df[df['y'].isin(['6175719.798486'])].navn

if df_xcoord.item() == df_ycoord.item():
    print('XY:', df_xcoord.item(),'\n')

# Multipoint
df_multipoint = df[df['wkb_geometry'].isin(['MULTIPOINT ((12.555485308174104 55.69383926601615))'])]
print('Multipoint:', df_multipoint.navn.item())

In [ ]:
# -------------------------------------------------------------------------------
# 5) Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.
# +
# 6) Gør kortet interaktiv så navnet på monumenterne vises når man trykker på et plot.
# -------------------------------------------------------------------------------

def mark_all_monuments():
    
    # Data frame indeholdende navne, samt koordinater osv.
    data = df[['navn', 'wkb_geometry', 'graffitirenhold']]
    
    # Frasortér alle monumenter uden navn
    data = data[data['navn'].notnull()]
    
    # Fjern paranteserne vha. regex
    data = data.replace(to_replace=r'\(\(', value='', regex=True)
    data = data.replace(to_replace=r'\)\)', value='', regex=True)
    
    # Udvid data frame med to nye kolonner for hhv. længde- og breddegrad
    new = data['wkb_geometry'].str.split(' ', n=2, expand=True)
    data['lon']= new[1]
    data['lat']= new[2]
    
    # Fjern multipoint kolonnen
    data.drop(columns =["wkb_geometry"], inplace = True)
    
    # Tilføj nye markøre
    for i in range(0,len(data)):
        
        # Sæt farve ud fra graffitirenhold, grøn=ja, rød=nej
        if data.iloc[i]['graffitirenhold'] == 'Ja':
            color = 'green'
        else:
            color = 'red'
            
        # Tilføj markør
        folium.Marker(
          location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
          popup=str(data.iloc[i]['navn']), icon=folium.Icon(color=color)
        ).add_to(map_osm)
    
    # Gem kortet til senere...
    map_osm.save('./osm.html')
    
    return map_osm

mark_all_monuments()

In [ ]:
# ----------------------------------------------------------------------------------------------------
# Samme som opgave 5+6, men bruger kolonnerne longitude og latitude fra csv, samt implementerer iframe
# ----------------------------------------------------------------------------------------------------

def mark_all_monumentsV2():
    
    # Frasortér alle monumenter uden navn
    data = df[df['navn'].notnull()]
    
    # Tilføj nye markøre
    for i in range(0,len(data)):
        
        html=f"""
        <h1> {data.iloc[i]['navn']}</h1>
        <p>
            Latitude: {data.iloc[i]['latitude']}<br>
            Longitude: {data.iloc[i]['longitude']}<br>
            Graffitirenhold: {data.iloc[i]['graffitirenhold']}
        </p>
        """
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        # Sæt farve ud fra graffitirenhold, grøn=ja, rød=nej
        if data.iloc[i]['graffitirenhold'] == 'Ja':
            color = 'green'
        else:
            color = 'red'
            
        # Tilføj markør
        folium.Marker(
          location=[data.iloc[i]['latitude'], data.iloc[i]['longitude']],
          popup=popup, icon=folium.Icon(color=color)
        ).add_to(map_osm)
    
    # Gem kortet til senere...
    map_osm.save('./osm.html')
    
    return map_osm

mark_all_monumentsV2()

https://kk.sites.itera.dk/apps/kk_monuments/index_liste.php